In [1]:
import pandas as pd
from selenium import webdriver as wd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import numpy as np
import time
import datetime
import re

<html>
<style>
        table, th, td {
        border:1px solid black;
        }
</style>
    <body>
        <h1>The Data Crawling Part</h1>
        <hr>
        <p>In this Part we are going to collect our data.</p>
        <p>The crawling is a selenium crawling from the 'www.il.kayak.com' website.</p>
        <p>The data will be as the table below:</p>
        <table>
        <tr>
            <th>Airline</th>
            <th>Date</th>
            <th>Source_Airport</th>
            <th>Destination_Airport</th>
            <th>Route</th>
            <th>Departure_Time</th>
            <th>Arrival_Time</th>
            <th>Duration</th>
            <th>Stops_Number</th>
            <th>Price</th>
        </tr>
        <tr>
            <th>...</th>
            <th>...</th>
            <th>...</th>
            <th>...</th>
            <th>...</th>
            <th>...</th>
            <th>...</th>
            <th>...</th>
            <th>...</th>
            <th>...</th>
        </tr>
        </table>
    </body>
</html>

<html>
    <body>
        <hr>
        <h>open_web</h>
        <hr>
        <p>Open web is a function that gets departure country(dep_country), the destination country(des_country), departure date(dep_date), ticket type (category) - there are three ticket search in the website - Cheapest/Best/Quickest
            and finally the web driver - we use the chrome driver.</p>
        <p></p>
        <p>This function builds a url from the parameters it gets, and commands the driver to open the webpage.</p>
        <p>**&fs=legdur=-600;airlines=-MULT,flylocal - defines the airlines not to be multiple, and that duration will be until 10h</p>
    </body>
</html>

In [2]:
def open_web(dep_country, des_country, dep_date, category,driver):
    url = ('https://www.il.kayak.com/flights/' + dep_country + '-' + des_country + '/' +
          dep_date + '/?sort=' + category + '&fs=legdur=-600'+ ';airlines=-MULT,flylocal')
    driver.get(url)

<html>
    <body>
        <hr>
        <h>more_button</h>
        <hr>
        <p>when we search for tickets in the kayak website there is a button 'Show more results', more_button function is a function that gets the clicking number on this button parameter(more_results_button) - that will command the driver click on this button 'number times'</p>
    </body>
</html>

In [3]:
def more_button(more_results_button,driver):
    for i in range(more_results_button):
        try:
            time.sleep(1.5)
            driver.find_element_by_xpath('//a[@class="moreButton"]').click()
        except:
            time.sleep(2)

<html>
    <body>
        <hr>
        <h>try_except_of_xpath</h>
        <hr>
        <p>When we need to get elements we need to the webpage be opened.</p>
        <p>This function gets xpath of the elements we must collect and two different times - normal and critical.</p>
        <p>the code will wait normal_time time, then gets the elements, sometimes the webpage takes more seconds to be loaded,
            so if the normal_time waiting is going to get error, we will wait another cirical_time time and then collect the elements.</p>
    </body>
</html>

In [4]:
def try_except_of_xpath(normal_time, critical_time, xpath, driver):
    try:
        time.sleep(normal_time)
        element = driver.find_elements_by_xpath(xpath)
    except:
        time.sleep(critical_time)
        element = driver.find_elements_by_xpath(xpath)
    finally:
        return element

<html>
    <body>
        <hr>
        <h>Get_Times</h>
        <hr>
        <p>When the webpage is opened we gets depart and arrival times element, collecting the text from them into two lists.</p>
        <p>.</p>
    </body>
</html>

In [5]:
def Get_Times(driver):
    Depart_elements = try_except_of_xpath(0.5,2,'\
    //span[@class="depart-time base-time"]',driver)
    Arrival_elements = try_except_of_xpath(0.5,2,'\
    //span[@class="arrival-time base-time"]',driver)
    Departure_Time = []
    Arrival_Time = []
    for i in range(len(Depart_elements)):
        Departure_Time.append(Depart_elements[i].text)
        Arrival_Time.append(Arrival_elements[i].text)
    return Departure_Time, Arrival_Time

<html>
    <body>
        <hr>
        <h>Get_Durations</h>
        <hr>
        <p>When the webpage is opened we gets duration element, collecting the text from them into two lists.</p>
    </body>
</html>

In [6]:
def Get_Durations(driver):
    durations = try_except_of_xpath(0.5,2, '\
    //div[@class="section duration allow-multi-modal-icons"]\
    /div[@class="top"]',driver)
    Duration = []
    for i in range(len(durations)):
        Duration.append(durations[i].text)
    return Duration

<html>
    <body>
        <hr>
        <h>Get_Airport_by_3_letters</h>
        <hr>
        <p>This function gets a raw test of an airport and takes the first three letters of it.</p>
        <p>For example: (raw_airport = 'TLV Ben Gurion int') and turns this into (airport = 'TLV')</p>
    </body>
</html>

In [7]:
def Get_Airport_by_3_letters(raw_airport):
    pattern = '(\w\w\w) .*'
    airport = re.findall(pattern,raw_airport)[0]
    return airport

<html>
    <body>
        <hr>
        <h>Get_Airports</h>
        <hr>
        <p>When the webpage is opened we get by one xpath the source and the destination airport elements. Then we must separate them into two lists and get the 3 first letters of the airport name (Get_Airport_by_3_letters function) .</p>
    </body>
</html>

In [8]:
def Get_Airports(driver):
    airports = try_except_of_xpath(0.5,2, '//span[@class="airport-name"]',driver)
    Source_Airports = []
    Destination_Airports = []
    for i in range(len(airports)//2):
        try:
            airport = airports[2*i].text
            Source_Airports.append(Get_Airport_by_3_letters(airport))
        except:
            Source_Airports.append(np.nan)
        try:
            airport = airports[1+2*i].text
            Destination_Airports.append(Get_Airport_by_3_letters(airport))
        except:
            Destination_Airports.append(np.nan)
    return Source_Airports, Destination_Airports

<html>
    <body>
        <hr>
        <h>Get_Stops</h>
        <hr>
        <p>When the webpage is opened we get the number of stops, and the stops elements. After this we collect the text from those elements.</p>
    </body>
</html>

In [9]:
def Get_Stops(driver):
    stops_num = try_except_of_xpath(0.5,2, '\
    //div[@class="section stops"]/div[@class="top"]',driver)
    stops_cities = try_except_of_xpath(0.5,2, '\
    //div[@class="section stops"]/div[@class="bottom"]',driver)
    Stops_Number = []
    Route = []
    for i in range(len(stops_num)):
        Stops_Number.append(stops_num[i].text)
        Route.append(stops_cities[i].text)
    return Stops_Number, Route

<html>
    <body>
        <hr>
        <h>Get_Price</h>
        <hr>
        <p>When the webpage is opened we get the Price elements. After this we collect the text from those elements.</p>
    </body>
</html>

In [10]:
def Get_Price(driver):
    ticket_price = try_except_of_xpath(0.5,2, '\
    //div[@class="booking"]//span[@class="price-text"]',driver)
    Price = []
    for ticket in ticket_price:
        Price.append(ticket.text[1:])
    return Price

<html>
    <body>
        <hr>
        <h>Get_Airlines</h>
        <hr>
        <p>When the webpage is opened we get the Airline elements. After this we collect the text from those elements.</p>
    </body>
</html>

In [11]:
def Get_Airlines(driver):
    airline = try_except_of_xpath(0.5,2,'\
    //div[@class="section codeshares allow-airlines"]/\
    span[@class="codeshares-airline-names"]',driver)
    Airlines = []
    for line in airline:
        Airlines.append(line.text)
    return Airlines

<html>
    <body>
        <hr>
        <h>crawl_page</h>
        <hr>
        <p>This function takes the the lists of the ticket data, by each function we described before. And returns a list of lists of the data we collected.</p>
    </body>
</html>

In [12]:
def crawl_page(driver):
    # getting list of the times of each flight (departure and arrival)
    time.sleep(1)
    Departure_Time, Arrival_Time = Get_Times(driver)
    # getting list of the durations of each flight
    time.sleep(1)
    Durations = Get_Durations(driver)
    # getting list of the airports we departure and arrive
    time.sleep(1)
    Source_Airport, Destination_Airport = Get_Airports(driver)
    # getting list of number and places of stops
    time.sleep(1)
    Stops_Number, Route = Get_Stops(driver)
    # getting the prices of the flights
    time.sleep(1)
    Price = Get_Price(driver)
    # getting the airlines of the flights
    time.sleep(1)
    Airlines = Get_Airlines(driver)
    return [Airlines, Source_Airport, Destination_Airport, Route, Departure_Time, Arrival_Time, Durations, 
           Stops_Number, Price]

<html>
    <body>
        <hr>
        <h>Write_CSV</h>
        <hr>
        <p>Sometimes the crawling data can be interupted and stoped by internet,electricity causes and etc...</p>
        <p>So it be easier to begin the process from the point we stopped and not to begin itself from the start. This why we save each page into our data</p>
        <p>This function recives a dictionary for the page tickets data we crawled, creates from it a df, and saves it with the other data we collected before.</p>
    </body>
</html>

In [13]:
def Write_CSV(normal_page_dic):
    try:
        Df = pd.DataFrame(normal_page_dic)
        Saved_DF = pd.read_csv('Crawled_Data.csv')
        New_df = pd.concat([Saved_DF,Df])
        New_df.to_csv('Crawled_Data.csv',index=False)
    except:
        Df.to_csv('Crawled_Data.csv',index=False)

<html>
    <body>
        <hr>
        <h>Crawl_data</h>
        <hr>
        <p>This function gets the departure countries as a list, the destination countries, the departure dates, and the more_results_button for the more_button function.</p>
        <p>We run on all of the option that can be between departure countries, destination countries and dates. For example:</p>
        <p>depart_countries = ['israel'], destination_countries = ['italy','denmark'], date = [29/2/2022].</p>
        <p>Our option are: israel->italy(29/2/2022),israel->denmark(29/2/2022)</p>
        <p>Then we run for each ticket category(Chepest/Best/Quickest) for each variation the algorithm below:</p>
        <p>step 1: open the webpage using open_web function.</p>
        <p>step 2: load more tickets in current webpage by clicking on the 'show more results' using more_button function.</p>
        <p>step 3: crawl and get the data from this page.</p>
        <p>step 4: if each list of the data is the same lengh use Write_CSV function to save the collected data as a dataframe.</p>
        <p>repeat the steps 1-4 for each variation, and restart the driver for each run (sometimes the websites wants the check if we are not robots. this prevents us from collecting the data.)</p>
    </body>
</html>

In [14]:
def Crawl_data(departure_countries ,countries ,dates, more_results_button):
    for dep_country in departure_countries:
        for country in countries:
            path = r'C:\Users\USER\Downloads\chromedriver_win32\chromedriver'
            driver = wd.Chrome(path)
            if country!=dep_country:
                for date in dates:
                    for category in ['price_a','bestflight_a','duration_a']:
                        try:
                            time.sleep(1)
                            open_web(dep_country, country,
                                     date, category, driver)
                        except:    
                            time.sleep(3)
                            open_web(dep_country, country,
                                     date, category, driver)
                        try:
                            time.sleep(2)
                            more_button(more_results_button, driver) 
                        except:
                            pass
                        try:
                            time.sleep(2)
                            page_info = crawl_page(driver)
                        except:
                            time.sleep(3)
                            page_info = crawl_page(driver)
                        try:
                            for x in page_info:
                                for y in page_info:
                                    if(len(x)!=len(y)):
                                        raise Exception('Wrong_Lenghts')
                            lengh = len(page_info[0])
                            dic = {
                                'Airline': page_info[0],
                                'Date':([date]*lengh),
                                'Source_Airport': page_info[1],
                                'Destination_Airport': page_info[2],
                                'Route': page_info[3],
                                'Departure_Time': page_info[4],
                                'Arrival_Time': page_info[5],
                                'Duration': page_info[6],
                                'Stops_Number': page_info[7],
                                'Price': page_info[8]
                            }
                            Write_CSV(dic)
                        except:
                            pass                        
            driver.close()
            time.sleep(1)

<html>
    <body>
        <hr>
        <h>The main run</h>
        <hr>
        <p>Run this line to start selenium crawling</p>
    </body>
</html>

In [ ]:
# 118cy = Italy, 90cy = Germany, 15cy = Austria 
# 225cy = Spain, 82cy = France, 94cy = Greece
# 24cy = Belgium, 174cy = Netherlands
# 252cy = UK, 65cy = Denmark
# 233cy = Switzerland
# 197cy = Portugal, 116cy = Ireland, 117cy = Israel
Crawl_data(['117cy'],
           ['118cy','90cy','15cy','225cy','82cy','94cy',
            '24cy','174cy','252cy','65cy','116cy','233cy'], 
            ['2022-02-06','2022-02-07','2022-02-08','2022-02-09','2022-02-10'
            ,'2022-02-11','2022-02-12','2022-03-06','2022-03-09','2022-03-12',
            '2022-03-27','2022-03-30','2022-04-01','2022-05-06',
            '2022-05-09','2022-05-12'], 3)

<ipython-input-14-ddfa8ea30e00>:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = wd.Chrome(path)
<ipython-input-3-067ee9cc7540>:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//a[@class="moreButton"]').click()
<ipython-input-4-33f877969e6e>:4: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  element = driver.find_elements_by_xpath(xpath)
